# Imports

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, TaskType
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    GenerationConfig,
    TrainingArguments, 
    Trainer
)
from trl import SFTTrainer

/home/leahheil/miniconda3/envs/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set Parameters

In [2]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training and evaluation
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
# max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

# ------------------------------------------------------------------------------------------------------------

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# ------------------------------------------------------------------------------------------------------------

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [3]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


# Load Models

In [4]:
device = "cuda:0"
model_name = "/home/leahheil/llama/weights"

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_special_tokens({"pad_token":"<pad>"})
tokenizer.padding_side = "left" #TODO changed it to left because generate told me to, have not ran it with left

# Load LLaMA model
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto", quantization_config=bnb_config)
model.config.pad_token_id = "<pad>"
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LoRA configuration
lora_config = LoraConfig(
    r=2, # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["q_proj","v_proj"]
)

model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=2)
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.47s/it]


trainable params: 1,048,576 || all params: 6,739,480,576 || trainable%: 0.015558706463730892


# Load Dataset

In [5]:
df_train = pd.read_csv("/home/leahheil/dataset/dataset-annotated-train.csv")
df_val = pd.read_csv("/home/leahheil/dataset/dataset-annotated-val.csv")
df_test = pd.read_csv("/home/leahheil/dataset/dataset-annotated-test.csv")

In [6]:
# drop indices and turn into dataset format
data_train = df_train.reset_index(drop=True)
data_test = df_test.reset_index(drop=True)
data_val = df_val.reset_index(drop=True)
data_train = Dataset.from_pandas(data_train)
data_test = Dataset.from_pandas(data_test)
data_val = Dataset.from_pandas(data_val)

# create the datasetdict object
data = DatasetDict()
data['train'] = data_train
data['test'] = data_test
data['val'] = data_val
data

DatasetDict({
    train: Dataset({
        features: ['text', 'prompt'],
        num_rows: 404
    })
    test: Dataset({
        features: ['text', 'prompt'],
        num_rows: 51
    })
    val: Dataset({
        features: ['text', 'prompt'],
        num_rows: 51
    })
})

In [7]:
def tokenize_function(example):
    prompt = [p for p in example["prompt"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt", max_length=4096).input_ids
    example['labels'] = tokenizer(example["text"], padding="max_length", truncation=True, return_tensors="pt", max_length=4096).input_ids
    return example

dataset = data.map(tokenize_function, batched=True)
dataset = dataset.remove_columns(['prompt', 'text'])

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 279.46 examples/s]


In [8]:
dataset_lengths = []
for i in dataset['val']:
    dataset_lengths.append(len(i['input_ids']))
print(dataset_lengths)

[4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096]


# Set up Trainer

In [10]:
# Set up peft trainer arguments
peft_training_args = TrainingArguments(
    output_dir="/home/leahheil/peft_output",
    evaluation_strategy="steps",
    save_strategy="epoch",
    # auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    group_by_length=True,
    fp16=fp16,
    bf16=bf16,
    eval_accumulation_steps=1,
    gradient_accumulation_steps=gradient_accumulation_steps,
    max_grad_norm=max_grad_norm,
)

#TODO: try parameters from https://colab.research.google.com/drive/1SYpgFpcmtIUzdE7pxqknrM4ArCASfkFQ?usp=sharing#scrollTo=cxc8sr_4-7DM

# Set up peft trainer
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
)


In [11]:
# Train model
peft_trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=404, training_loss=2.083915937064898, metrics={'train_runtime': 6288.8611, 'train_samples_per_second': 0.064, 'train_steps_per_second': 0.064, 'total_flos': 3.3463681085865984e+16, 'train_loss': 2.083915937064898, 'epoch': 1.0})

In [52]:
# Save trained model
peft_model_path="/home/leahheil/notebooks/peft_output/checkpoint-404"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

# trained_model = AutoModelForSeq2SeqLM.from_pretrained("/home/leahheil/peft_output/checkpoint-TODO", torch_dtype=torch.bfloat16).to(device)


('/home/leahheil/notebooks/peft_output/checkpoint-404/tokenizer_config.json',
 '/home/leahheil/notebooks/peft_output/checkpoint-404/special_tokens_map.json',
 '/home/leahheil/notebooks/peft_output/checkpoint-404/tokenizer.json')

In [54]:
trained_peft_model = PeftModel.from_pretrained(model, peft_model_path)

In [55]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 404
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 51
    })
    val: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 51
    })
})

In [56]:
prompts = data['test'][0:10]['prompt']
baseline_job_ads = data['test'][0:10]['text']
model.config.pad_token_id = model.config.eos_token_id

original_model_job_ads = []
# instruct_model_job_ads = []
peft_model_job_ads = []

for idx, prompt in enumerate(prompts):
    input_ids = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt", max_length=4096).input_ids.to(device)

    human_baseline_text_output = baseline_job_ads[idx]
    
    original_model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=4096, pad_token_id=0))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    # instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=4096))
    # instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = trained_peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=4096, pad_token_id=0))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_job_ads.append(original_model_text_output)
    # instruct_model_job_ads.append(instruct_model_text_output)
    peft_model_job_ads.append(peft_model_text_output)

zipped_summaries = list(zip(baseline_job_ads, original_model_job_ads, peft_model_job_ads))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline', 'original_model', 'peft_model'])
df

,human_baseline,original_model,peft_model
0,Wat je gaat doen․ Vervoer je graag bouwmateria...,Schrijf een vacature text voor Chauffeur bouwm...,Schrijf een vacature text voor Chauffeur bouwm...
1,Wat je gaat doen Wil jij werken als procesoper...,Schrijf een vacature text voor Procesoperator ...,Schrijf een vacature text voor Procesoperator ...
2,Wat je gaat doen Ben je technisch onderlegd en...,Schrijf een vacature text voor Operator Medica...,Schrijf een vacature text voor Operator Medica...
3,Wat je gaat doen․ Zoek jij een baan met regelm...,Schrijf een vacature text voor Chauffeur koelt...,Schrijf een vacature text voor Chauffeur koelt...
4,Vacatureomschrijving․ Matchpartner Onderwijs z...,Schrijf een vacature text voor Docent Nederlan...,Schrijf een vacature text voor Docent Nederlan...
5,Wat je gaat doen FrieslandCampina heeft een va...,Schrijf een vacature text voor Analyst Packagi...,Schrijf een vacature text voor Analyst Packagi...
6,Wat je gaat doen Lekker werken in de buitenluc...,Schrijf een vacature text voor Chauffeur vuiln...,Schrijf een vacature text voor Chauffeur vuiln...
7,Vacatureomschrijving․ Graduate Class Software ...,Schrijf een vacature text voor Graduate Class ...,Schrijf een vacature text voor Graduate Class ...
8,Wat je gaat doen Wil jij lekker in de buitenlu...,Schrijf een vacature text voor Fulltime Magazi...,Schrijf een vacature text voor Fulltime Magazi...
9,Vacatureomschrijving. De commercieel directeur...,Schrijf een vacature text voor Commercieel Dir...,Schrijf een vacature text voor Commercieel Dir...


In [57]:
print(df["peft_model"][0])

Schrijf een vacature text voor Chauffeur bouwmaterialen. Noem in iedergeval de volgende aspecten: 
- De functie omschrijving: Als chauffeur bouwmaterialen ben je verantwoordelijk voor het vervoeren van goederen naar klanten in de regio. Je laadt en lost de goederen en zorgt ervoor dat ze veilig worden vervoerd. Daarnaast heb je ook contact met de klanten en handel je het administratieve gedeelte af. De werkdag begint vroeg en eindigt in de middag, waardoor je een goede balans hebt tussen werk en privé.
- De benodigde vaardigheden: Vereiste vaardigheden voor deze functie zijn een geldige bestuurderskaart, CE-rijbewijs met Code 95, ervaring met een kooi-aap of heftruck, ervaring met een trekker-oplegger en wonen in de omgeving van Den Haag, Rotterdam of Delft.
- De arbeidsvoorwarden: Als chauffeur bouwmaterialen bij Start People Transport ontvang je een salaris tussen €2.100 en €2.700 bruto per maand op basis van ervaring. Daarnaast krijg je persoonlijke begeleiding, vakantiegeld, opbouw

In [58]:
print(model.config.pad_token_id)

2


In [60]:
def generate(prompt):
    inputs = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt", max_length=4096)
    input_ids = inputs["input_ids"].to(device)
    generation_output = trained_peft_model.generate(
            input_ids=input_ids,
            generation_config=GenerationConfig(temperature=1.0, top_p=1.0, top_k=50, num_beams=1, pad_token_id=2),
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=2048
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        print(output)
generate(data["test"][0]["prompt"])

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> Schrijf een vacature text voor Chauffeur bouwmaterialen. Noem in iedergeval de volgende aspecten: 
- De functie omschrijving: Als chauffeur bouwmaterialen ben je verantwoordelijk voor het vervoeren van goederen naar klanten in de regio. Je laadt en lost de goederen en zorgt ervoor dat ze veilig worden vervoerd. Daarnaast heb je ook contact met de klanten en handel je het administratieve gedeelte af. De werkdag begint vroeg en eindigt in de middag, waardoor je een goede balans hebt tussen werk en privé.
- De benodigde vaardigheden: Vereiste vaardigheden voor deze functie zijn een geldige bestuurderskaart, CE-rijbewijs met Code 95, ervaring met een kooi-aap of heftruck, ervaring met een trekker-oplegger en wonen in de omgeving van Den Haag, Rotterdam of Delft.
- De arbeidsvoorwarden: Als chauffeur bouwmaterialen bij Start People Transport ontvang je een salaris tussen €2.100 en €2.700 bruto per maand op basis van ervaring. Daarnaast krijg je persoonlijke begeleiding, vakantiegeld, op

In [62]:
from transformers import pipeline

pipe = pipeline(model="/home/leahheil/notebooks/peft_output/checkpoint-404", return_full_text=False)
print(pipe(data["test"][0]["prompt"]))

RuntimeError: Instantiating a pipeline without a task set raised an error: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/home/leahheil/llama/weights'. Use `repo_type` argument if needed.